# Multivariate Fleet Analysis

_Dataset_: Supplementary data for Megill et al. (2024): "Alternative climate metrics to the Global Warming Potential are more suitable for assessing aviation non-CO2 effects"

_Authors_:

- Liam Megill (1, 2), https://orcid.org/0000-0002-4199-6962   
- Kathrin Deck (2)  
- Volker Grewe (1, 2), https://orcid.org/0000-0002-8012-6783  

_Affiliation (1)_: Deutsches Zentrum für Luft- und Raumfahrt (DLR), Institut für Physik der Atmosphäre, Oberpfaffenhofen, Germany

_Affiliation (2)_: Delft University of Technology (TU Delft), Faculty of Aerospace Engineering, Section Aircraft Noise and Climate Effects (ANCE), Delft, The Netherlands

_Corresponding author_: Liam Megill, liam.megill@dlr.de

_doi_: https://doi.org/10.1038/s43247-024-01423-6

---


### Summary
This notebook analyses the data generated by AirClim for the fleets in the Monte Carlo simulation. It corresponds to the results shown for "REQ 1: Climate metric neutrality with respect to aviation emissions" in the linked paper, in particular Figure 1 and Extended Data Figure 1. The fleets are shown in MCMV_init.txt and are created using MVMC_init.py. The fleet fuel scenarios used can be found in E_bg_fuel_MC.txt.

### Linked data
- `MVMC/batch_*`: Due to memory limitations, the total 10000 fleets were divided into 10 batches of 1000 fleets. Each fleet has its own folder, Fleet1 to Fleet1003. Fleets 1 to 3 are the reference low, medium and high fleet scenarios
- `MVMC/AGWPH_CO2_SSP2-45.txt`: AGWP of a CO2 pulse with the SSP2-4.5 scenario for time horizons H between 1 and 100 years
- `MVMC/E_bg_fuel_MC.txt`: Fleet fuel scenarios used as input into AirClim
- `MVMC/MVMC_init.py`: Python script that performs the Monte Carlo simulation and creates the 10000 fleets
- `MVMC/MVMC_init.txt`: Result of the Monte Carlo simulation: parameters that define the 10000 fleets, later used as an input into AirClim.
- `MVMC/tot_err_arr_*.npy`: Array of incorrect fleet pairings as a function of time horizon (5-year steps) for peak temperature (maxT) and average temperatures over 20, 50 and 100 years (av*).

---

### Copyright

Copyright © 2024 Liam Megill

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0. Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

## Setup

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.gridspec import GridSpec
import seaborn as sns

# define plotting style
colors = ['#2271B2', '#3DB7E9', '#F748A5', '#359B73', '#D55E00', '#E69F00', '#F0E442']
sns.set_context("paper", rc={"lines.linewidth": 2})
sns.set_style("ticks")

## Function definitions

The climate metric calculations are defined in Table 2 of the linked paper.

In [ ]:
def load_rf_dt(directory):
    """Load the radiative forcing (RF) and temperature change (dT) responses of a given fleet.

    Args:
        directory (_str_): Path to data directory.

    Returns:
        rf_arr (_np.ndarray_): Array (7, len(years)) of radiative forcing results for each emission species
        dt_arr (_np.ndarray_): Array (7, len(years)) of temperature change results for each emission species
        years (_np.ndarray_): Array of response years from AirClim
    """
    emission_species = ["CO2", "H2O", "O3", "CH4", "Cont", "PMO"]
    years = np.loadtxt(directory + "RF_CO2_taumean_rfmean.txt", skiprows=2, usecols=0)
    rf_arr = np.zeros((7, len(years)))  # define empty RF array
    dt_arr = np.zeros((7, len(years)))  # define empty dT array
    # populate rf_arr and dt_arr
    for i in range(len(emission_species)):
        rf_name = "RF_" + emission_species[i] + "_taumean_rfmean.txt"
        dt_name = "dt_" + emission_species[i] + "_taumean_rfmean_lammean.txt"
        rf_lst = np.loadtxt(directory + rf_name, skiprows=2, usecols=1)  # uses default AirClim output format
        dt_lst = np.loadtxt(directory + dt_name, skiprows=2, usecols=1)  # uses default AirClim output format
        rf_arr[i + 1, :] = rf_lst  # i+1 because i=0 is the total
        dt_arr[i + 1, :] = dt_lst
    # Add totals
    rf_arr[0, :] = np.sum(rf_arr[:, :], axis=0)
    dt_arr[0, :] = np.sum(dt_arr[:, :], axis=0)
    return rf_arr, dt_arr, years


def rf(H, rf_arr, year, years):
    """Calculates the Radiative Forcing (RF) as a climate metric for the time horizon H

    Args:
        H (_float_): Time horizon [years]
        rf_arr (_np.ndarray_): Array (7, len(years)) of RF results for each emission species and total
        year (_float_): Year for which RF is to be calculated
        years (_np.ndarray_): Array of response years from AirClim

    Returns:
        _float_: RF_H(year)
    """
    return rf_arr[0, int(year - years[0] + H)]


def agwp(H, rf_arr, year, years):
    """Calculates the Absolute Global Warming Potential (AGWP) for the time horizon H

    Args:
        H (_float_): Time horizon [years]
        rf_arr (_np.ndarray_): Array (7, len(years)) of RF results for each emission species and total
        year (_float_): Year for which AGWP is to be calculated
        years (_np.ndarray_): Array of response years from AirClim

    Returns:
        _float_: AGWP_H(year)
    """
    agwp_lst = np.array([[np.sum(rf_arr[j, i:i + H]) for i in range(len(years) - H)] for j in range(7)])
    return agwp_lst[0, int(year - years[0])]


def aegwp(H, rf_arr, year, years):
    """Calculates the Absolute Effective Global Warming Potential (AEGWP) for the time horizon H

    Args:
        H (_float_): Time horizon [years]
        rf_arr (_np.ndarray_): Array (7, len(years)) of RF results for each emission species and total
        year (_float_): Year for which AEGWP is to be calculated
        years (_np.ndarray_): Array of response years from AirClim

    Returns:
        _float_: AEGWP_H(year)
    """
    
    efficacies = np.array([1., 1.14, 1.37, 1.18, 0.59, 1.])  # efficacies from Ponater et al. (2006)
    aegwp_lst = np.array([[np.sum(rf_arr[j, i:i + H]) for i in range(len(years) - H)] for j in range(7)]) 
    aegwp_lst[1:, :] = aegwp_lst[1:, :] * efficacies.reshape((6, 1))
    aegwp_lst[0, :] = np.sum(aegwp_lst[1:, :], axis=0)
    return aegwp_lst[0, int(year - years[0])]


def agtp(H, dt_arr, year, years):
    """Calculates the Absolute Global Temperature-Change Potential (AGTP) for the time horizon H

    Args:
        H (_float_): Time horizon [years]
        dt_arr (_np.ndarray_): Array (7, len(years)) of dT results for each emission species and total
        year (_float_): Year for which AGTP is to be calculated
        years (_np.ndarray): Array of response years from AirClim

    Returns:
        _float_: AGTP_H(year)
    """
    return dt_arr[0, int(year - years[0] + H)]


def iagtp(H, dt_arr, year, years):
    """Calculates the integrated Absolute Temperature-Change Potential (iAGTP) for the time horizon H

    Args:
        H (_float_): Time horizon [years]
        dt_arr (_np.ndarray_): Array (7, len(years)) of dT results for each emission species and total
        year (_float_): Year for which iAGTP is to be calculated
        years (_np.ndarray): Array of response years from AirClim
    
    Returns:
        _float_: iAGTP_H(year)
    """
    iagtp_lst = np.array([[np.sum(dt_arr[j, i:i + H]) for i in range(len(years) - H)] for j in range(7)])
    return iagtp_lst[0, int(year - years[0])]


def atr(H, dt_arr, year, years):
    """Calculates the absolute Average Temperature Response (ATR) for the time horizon H

    Args:
        H (_float_): Time horizon [years]
        dt_arr (_np.ndarray_): Array (7, len(years)) of dT results for each emission species and total
        year (_float_): Year for which ATR is to be calculated
        years (_np.ndarray): Array of response years from AirClim

    Returns:
        _float_: ATR_H(year)
    """
    atr_lst = np.array([[np.sum(dt_arr[j, i:i + H]) / H for i in range(len(years) - H)] for j in range(7)])
    return atr_lst[0, int(year - years[0])]


def gwps(H, rf_arr, year, years, co2_lst, agwph_co2_file):
    """Calculates the GWP* climate metric for the time horizon H

    Args:
        H (_float_): Time horizon [years]
        rf_arr (_float_): Array (7, len(years)) of RF results for each emission species and total
        year (_float_): Year for which GWP* is to be calculated
        years (_np.ndarray_): Array of response years from AirClim
        co2_lst (_np.ndarray_): Array (1, len(years)) of yearly CO2 emissions 
        agwph_co2_file (_str_): Path to AGWP_CO2(H) file (AGWP as a function of H for CO2 for 1<=H<=100)

    Returns:
        _float_: GWP*_H(year)
    """
    dt = 20
    s = 0.25  # in line with Smith et al. (2021) for CH4
    agwph_co2 = np.loadtxt(agwph_co2_file)
    yr_idx1 = np.arange(dt, len(rf_arr[0, :]))
    agwp_co2 = agwph_co2[H-1] * 1e9
    dFdt = np.subtract(rf_arr[:, np.arange(dt, len(years))], rf_arr[:, np.arange(0, len(years) - dt)]) / dt / 1000.
    F_av = np.array([[sum(rf_arr[j, i - dt + 1: i + 1]) / dt for i in np.arange(dt, len(years))] for j in range(7)]) / 1000.
    g = (1 - np.exp(-s / (1 - s))) / s  # Extension by Smith et al. (2021)
    gwps_co2eq = g * ((1 - s) * dFdt * H / agwp_co2 + s * F_av / agwp_co2)
    gwps_co2eq[1, :] = co2_lst[yr_idx1]  # Add co2 emissions directly
    gwps_co2eq[0, :] = np.sum(gwps_co2eq[1:, :], axis=0)
    return max(gwps_co2eq[0, :])


def egwps(H, rf_arr, year, years, co2_lst, agwph_co2_file):
    """Calculates the EGWP* climate metric for the time horizon H

    Args:
        H (_float_): Time horizon [years]
        rf_arr (_float_): Array (7, len(years)) of RF results for each emission species and total
        year (_float_): Year for which GWP* is to be calculated
        years (_np.ndarray_): Array of response years from AirClim
        co2_lst (_np.ndarray_): Array (1, len(years)) of yearly CO2 emissions 
        agwph_co2_file (_str_): Path to AGWP_CO2(H) file (AGWP as a function of H for CO2 for 1<=H<=100)

    Returns:
        _float_: EGWP*_H(year)
    """
    dt = 20
    s = 0.25  # in line with Smith et al. (2021) for CH4
    efficacies = np.array([1., 1.14, 1.37, 1.18, 0.59, 1.])  # efficacies from Ponater et al. (2006)
    agwph_co2 = np.loadtxt(agwph_co2_file)
    yr_idx1 = np.arange(dt, len(rf_arr[0, :]))
    agwp_co2 = agwph_co2[H-1] * 1e9
    dFdt = np.subtract(rf_arr[:, np.arange(dt, len(years))], rf_arr[:, np.arange(0, len(years) - dt)]) / dt / 1000.
    F_av = np.array([[sum(rf_arr[j, i - dt + 1: i + 1]) / dt for i in np.arange(dt, len(years))] for j in range(7)]) / 1000.
    g = (1 - np.exp(-s / (1 - s))) / s  # Extension by Smith et al. (2021)
    gwps_co2eq = g * ((1 - s) * dFdt * H / agwp_co2 + s * F_av / agwp_co2)
    gwps_co2eq[1, :] = co2_lst[yr_idx1]  # Add co2 emissions directly
    gwps_co2eq[1:, :] = gwps_co2eq[1:, :] * efficacies.reshape((6, 1))
    gwps_co2eq[0, :] = np.sum(gwps_co2eq[1:, :], axis=0)
    return max(gwps_co2eq[0, :])


def calc_fleet_pairing_err(met_arr, full_data, T_arr, H_lst, i_co):
    """Calculates the number of incorrect fleet pairings.

    Args:
        met_arr (_np.ndarray_): Array (len(full_data[:, 0]), len(H_lst)) of climate metric values
        full_data (_np.ndarray_): Array of full fleet data names
        T_arr (_np.ndarray_): Array (len(full_data[:, 0]), 4) of climate objectives
        H_lst (_np.ndarray_): Array of time horizons
        i_co (_int_): Climate objective index: 0 - max(dT); 1 - 20-yr average; 2 - 50-yr average; 3 - 100-yr average

    Returns:
        tot_err_lst (_np.ndarray_): Array of incorrect fleet pairing percentage per time horizon.
    """
    tot_err_lst = np.empty(len(H_lst))
    for i_H in range(len(H_lst)):
        M_err_arr = np.empty((len(full_data[:, 0]), len(full_data[:, 0])))
        for n in range(len(full_data[:, 0])):
            for m in range(len(full_data[:, 0])):
                T_diff = T_arr[m, i_co] - T_arr[n, i_co]
                CM_diff = met_arr[m, i_H] - met_arr[n, i_H]
                M_err_arr[n, m] = T_diff * CM_diff
        tot_err_lst[i_H] = len(np.where(M_err_arr < 0.)[0]) / (len(full_data[:, 0]) ** 2) * 100.
    return tot_err_lst

## Read fleet data and calculate climate metric (H=100) responses

Due to memory limitations encountered in the running of multiple AirClim instances, the 10.000 runs were split into 10 divisions (batch_1 to batch_10). In this block, the data from all AirClim runs is read. For each fleet, the climate metric responses are calculated for a time horizon of 100 years (metric_arr). Also calculated are the climate objectives (T_arr) - maximum dT and average dT over 20, 50 and 100 years.

Beware: the calculations in this section can take a while to complete (~30 mins on high-end laptop for all batches). To save time, the previously calculated metric_arr and T_arr arrays can instead be read from file.

In [ ]:
use_saved_arrays = True

# read data
versions = ["../data/MVMC/batch_%s" % i for i in range(1, 11)]  # up to 11
full_data = np.empty((len(versions) * 1000 + 3, 10), dtype="object")
fleet_load = np.empty((len(versions) * 1000 + 3, 2), dtype="object")

for i, version in enumerate(versions):
    data = np.loadtxt(version + "/MVMC_init.txt", skiprows=1)
    if i == 0:
        full_data[0:1003, :] = data
        fleet_load[0:1003, 0] = ['Fleet%s' % int(num) for num in data[:, 0]]
        fleet_load[0:1003, 1] = ['%s' % version for j in range(len(data[:, 0]))]
    else:
        full_data[i * 1000 + 3:(i+1) * 1000 + 3, :] = data[3:, :]
        fleet_load[i*1000+3:(i+1)*1000+3, 0] = ['Fleet%s' % int(num) for num in data[3:, 0]]
        fleet_load[i*1000+3:(i+1)*1000+3, 1] = ['%s' % version for j in range(len(data[3:, 0]))]

if use_saved_arrays:
    metric_arr = np.load("../data/MVMC/metric_arr.npy")
    T_arr = np.load("../data/MVMC/T_arr.npy")

else:
    # initialise arrays
    metric_arr = np.empty((len(full_data[:, 0]), 7))
    T_arr = np.empty((len(full_data[:, 0]), 4))

    # calculate climate metric and climate objective for each fleet
    for i in range(len(full_data[:, 0])):
        if i % 1000 == 0:
            print('Idx %s' % i)
        analysis_year = int(full_data[i, 1])
        folder = "%s/%s/" % (fleet_load[i, 1], fleet_load[i, 0])
        rf_arr, dt_arr, years = load_rf_dt(folder)
        year_idx = int((np.where(years == analysis_year))[0])
        co2_emis = (np.loadtxt(folder + "CO2_emis.txt"))[:, 1]
        metric_arr[i, 0] = rf(100, rf_arr, analysis_year, years)
        metric_arr[i, 1] = agwp(100, rf_arr, analysis_year, years)
        metric_arr[i, 2] = aegwp(100, rf_arr, analysis_year, years)
        metric_arr[i, 3] = agtp(100, dt_arr, analysis_year, years)
        metric_arr[i, 4] = atr(100, dt_arr, analysis_year, years)
        metric_arr[i, 5] = gwps(100, rf_arr, analysis_year, years, co2_emis, "../data/MVMC/AGWPH_CO2_SSP2-45.txt")
        metric_arr[i, 6] = egwps(100, rf_arr, analysis_year, years, co2_emis, "../data/MVMC/AGWPH_CO2_SSP2-45.txt")
        T_arr[i, 0] = max(dt_arr[0, :])
        T_arr[i, 1] = np.average(dt_arr[0, year_idx:year_idx + 21])
        T_arr[i, 2] = np.average(dt_arr[0, year_idx:year_idx + 51])
        T_arr[i, 3] = np.average(dt_arr[0, year_idx:year_idx + 101])
    
    # to save metric_arr and T_arr, use np.save("[...].npy")

## Plot dCM vs dT

This section plots the pairwise comparison of the climate metric change and the temperature change of all given fleet pairings (Extended Data Figure 1 in linked paper). Uncomment the last line to save the figure to the `images` folder.

Beware: this section can take a while to complete.

In [ ]:
i_co = 0  # climate objective index in T_arr (0 - peak T; 1 - 20yr av.; 2 - 50yr av.; 3 - 100yr av.)

# initialise figure
fig, axs = plt.subplots(2, 4, figsize=(16, 8), sharex="all", sharey="all")
metric_names = ["RF", "AGWP", "AEGWP", "AGTP", "iAGTP & ATR", "GWP*", "EGWP*"]
alpha_colors = ["#F8FAFC", "#F9FCFE", "#FEF9FC", "#F8FCFA", "#FDFAF7", "#FEFCF7", "#FEFEF9"]

# plot metric responses
for i in range(7):
    ax = axs.flatten()[i]
    x = np.array([T_arr[:, i_co] - T_arr[k, i_co] for k in range(len(T_arr))]).flatten()  # difference in climate objective
    y = np.array([(metric_arr[:, i] - metric_arr[k, i]) / (max(metric_arr[:, i]) - min(metric_arr[:, i])) for k in range(len(metric_arr[:, 0]))]).flatten()

    # Transparencies range
    transparency_start = 0.03
    transparency_end = 1.0

    # Create a colormap dictionary with transparency values
    hex_color = colors[i]
    cmap_dict = {
        'red':   [(0.0, float(int(hex_color[1:3], 16)) / 255, float(int(hex_color[1:3], 16)) / 255),
                (1.0, float(int(hex_color[1:3], 16)) / 255, float(int(hex_color[1:3], 16)) / 255)],
        'green': [(0.0, float(int(hex_color[3:5], 16)) / 255, float(int(hex_color[3:5], 16)) / 255),
                (1.0, float(int(hex_color[3:5], 16)) / 255, float(int(hex_color[3:5], 16)) / 255)],
        'blue':  [(0.0, float(int(hex_color[5:], 16)) / 255, float(int(hex_color[5:], 16)) / 255),
                (1.0, float(int(hex_color[5:], 16)) / 255, float(int(hex_color[5:], 16)) / 255)],
        'alpha': [(0.0, transparency_start, transparency_start),
                (1.0, transparency_end, transparency_end)]
    }

    # Create the colormap
    cmap = LinearSegmentedColormap('custom_cmap', cmap_dict)
    
    # Plot 
    ax.axvline(0, color='k', linestyle='-', linewidth=0.5, alpha=0.5)
    ax.axhline(0, color='k', linestyle='-', linewidth=0.5, alpha=0.5)
    sns.scatterplot(x=x, y=y, s=1, color=alpha_colors[i], ax=ax)
    ax.hexbin(x, y, gridsize=100, mincnt=10, cmap=cmap, linewidths=0.1)
    ax.text(0.95, 0.05, metric_names[i], transform=ax.transAxes, ha='right', va='bottom', fontsize=14)
    ax.tick_params(axis='x', labelsize=12)
    ax.tick_params(axis='y', labelsize=12)


# plot explanatory image
ax = axs.flatten()[-1]
ax.axvline(0, color='k', linestyle='-', linewidth=0.5, alpha=0.5)
ax.axhline(0, color='k', linestyle='-', linewidth=0.5, alpha=0.5)
ax.fill_between([-12, 0], 0, 1.2, color="none", edgecolor="#db5856", hatch="/", alpha=0.7)
ax.fill_between([0, 12], -1.2, 0, color="none", edgecolor="#db5856", hatch="/", alpha=0.7)
ax.plot([-10.5, 10.5], [-1.2, 1.2], "gray", linestyle='--', linewidth=1)
ax.text(0.05, 0.95, "$\Delta$CM > 0 but \n $\Delta$T < 0", transform=ax.transAxes, ha='left', va='top', bbox=dict(facecolor="white", alpha=1, edgecolor="#db5856"))
ax.text(0.95, 0.05, "$\Delta$CM < 0 but \n $\Delta$T > 0", transform=ax.transAxes, ha='right', va='bottom', bbox=dict(facecolor="white", alpha=1, edgecolor="#db5856"))
ax.text(0.85, 0.8, "Ideal", transform=ax.transAxes, rotation=45, fontsize=12)
ax.plot(0, 0, colors[1], marker="o", markersize=10)
ax.text(-0.5, -0.1, "A", color=colors[1], fontsize=12, ha='right', va='top', bbox=dict(facecolor="white", alpha=0.9))
ax.plot(-4, 0.6, "#db5856", marker="o", markersize=10)
ax.text(-4.5, 0.5, "B", color="#db5856", fontsize=12, ha='right', va='top', bbox=dict(facecolor="white", alpha=0.9))
ax.plot(6, 0.4, colors[3], marker="o", markersize=10)
ax.text(5.5, 0.3, "C", color=colors[3], fontsize=12, ha='right', va='top', bbox=dict(facecolor="white", alpha=0.9))

# set axis sizes
ax.tick_params(axis='x', labelsize=12)
ax.set_yticks([-1.0, 0.0, 1.0])
ax.set_xticks([-8, -4, 0, 4, 8])
ax.set_xlim([-10.5, 10.5])
ax.set_ylim([-1.2, 1.2])

# Adjust layout and save
plt.tight_layout()
fig.text(0.53, 0.02, 'Difference in peak temperature $\Delta$T between fleets [mK]', ha='center', fontsize=14)
fig.text(0.02, 0.5, 'Normalised absolute climate metric difference $\Delta$CM between fleets [-]', va='center', rotation='vertical', fontsize=14)
fig.subplots_adjust(left=0.08, right=0.98, bottom=0.08, top=0.98)
fig_savename = "../images/pairwise_dTpeak.png"  # define savename with path
# fig.savefig(fig_savename, dpi=600)

## Calculate dependence of the incorrect fleet pairings on the time horizon

The above results are valid for a single time horizon H. The next two sections calculate the dependence of the incorrect fleet pairings (those in the top left and bottom right quadrants in the above plot) on the time horizon for each climate metric. This section uses full_data, as calculated previously, to read the data and calculate the climate metric responses for a range of time horizons (H_lst).

Calculating the dependence on the time horizon is computationally very intense (~6 hours on high-end laptop for all batches for a 5-year H step). Making H_lst finer dramatically increases the computational time. Therefore, saved data is also available in the `MVMC` folder. Use the `use_saved_results` boolean to load the data from file, make sure to select the right path in line 4.

In [ ]:
use_saved_results = True

# if saved data is to be used
if use_saved_results:
    H_lst = np.arange(5, 101, 5)
    err_arr = np.load("../data/MVMC/tot_err_arr_maxT.npy")

# if error array is to be calculated (beware: very long calculation time!)
else:
    i_co = 0  # Climate objective index: 0 - max(dT); 1 - 20-yr average; 2 - 50-yr average; 3 - 100-yr average
    # initialise arrays
    H_lst = np.arange(20, 101, 20)  # beware: reducing the step dramatically increases the computational time
    rf_met_arr = np.empty((len(full_data[:, 0]), len(H_lst)))
    agwp_arr = np.empty((len(full_data[:, 0]), len(H_lst)))
    aegwp_arr = np.empty((len(full_data[:, 0]), len(H_lst)))
    agtp_arr = np.empty((len(full_data[:, 0]), len(H_lst)))
    atr_arr = np.empty((len(full_data[:, 0]), len(H_lst)))
    gwps_arr = np.empty((len(full_data[:, 0]), len(H_lst)))
    egwps_arr = np.empty((len(full_data[:, 0]), len(H_lst)))

    # load data and calculate metric values as a function of time horizon
    for i_fl in range(len(full_data[:, 0])):
        if i_fl % 1000 == 0:
            print("Idx %s" % i_fl)
        analysis_year = int(full_data[i_fl, 1])
        folder = "%s/%s/" % (fleet_load[i_fl, 1], fleet_load[i_fl, 0])
        rf_arr, dt_arr, years = load_rf_dt(folder)
        year_idx = int((np.where(years == analysis_year))[0])
        co2_emis = (np.loadtxt(folder + "CO2_emis.txt"))[:, 1]

        for i_H, H in enumerate(H_lst):
            rf_met_arr[i_fl, i_H] = rf(H, rf_arr, analysis_year, years)
            agwp_arr[i_fl, i_H] = agwp(H, rf_arr, analysis_year, years)
            aegwp_arr[i_fl, i_H] = aegwp(H, rf_arr, analysis_year, years)
            agtp_arr[i_fl, i_H] = agtp(H, dt_arr, analysis_year, years)
            atr_arr[i_fl, i_H] = atr(H, dt_arr, analysis_year, years)
            gwps_arr[i_fl, i_H] = gwps(H, rf_arr, analysis_year, years, co2_emis, "../data/MVMC/AGWPH_CO2_SSP2-45.txt")
            egwps_arr[i_fl, i_H] = egwps(H, rf_arr, analysis_year, years, co2_emis, "../data/MVMC/AGWPH_CO2_SSP2-45.txt")

    err_arr = np.empty((len(H_lst), 7))
    err_arr[:, 0] = calc_fleet_pairing_err(rf_met_arr, full_data, T_arr, H_lst, i_co)
    err_arr[:, 1] = calc_fleet_pairing_err(agwp_arr, full_data, T_arr, H_lst, i_co)
    err_arr[:, 2] = calc_fleet_pairing_err(aegwp_arr, full_data, T_arr, H_lst, i_co)
    err_arr[:, 3] = calc_fleet_pairing_err(agtp_arr, full_data, T_arr, H_lst, i_co)
    err_arr[:, 4] = calc_fleet_pairing_err(atr_arr, full_data, T_arr, H_lst, i_co)
    err_arr[:, 5] = calc_fleet_pairing_err(gwps_arr, full_data, T_arr, H_lst, i_co)
    err_arr[:, 6] = calc_fleet_pairing_err(egwps_arr, full_data, T_arr, H_lst, i_co)
    
    # save err_arr
    # err_arr_savename = "../data/MVMC/tot_err_arr_*.npy" (maxT, av20, av50, av100)
    # np.save(err_arr_savename, err_arr)

## Plot incorrect fleet pairings vs time horizon

This section plots the incorrect fleet pairings against the time horizon. Uncomment the last line to save the figure to the `images` folder.

In [ ]:
fig_err, ax_err = plt.subplots(figsize=(6, 4))
metrics = ["RF", "AGWP", "AEGWP", "AGTP", "ATR & iAGTP", "GWP*", "EGWP*"]
markers = ["D", "o", "s", "h", "^", "*", "X"]
linestyles = ['-', '--', '--', '-.', ':', (0, (3, 1, 1, 1, 1, 1)), (0, (3, 1, 3, 1, 1, 1))]
markersizes = [6, 6, 6, 6, 6, 8, 7]
for i in range(len(metrics)):
    ax_err.plot(H_lst, err_arr[:, i], color=colors[i], linestyle=linestyles[i], linewidth=2, label=metrics[i], marker=markers[i], markersize=markersizes[i])
ax_err.set_xlabel("Time Horizon [yr]", fontsize=12)
ax_err.set_ylabel("Total error frequency [%]", fontsize=12)
ax_err.set_xlim([-3, 103])
ax_err.set_ylim([0, 8])
ax_err.tick_params(axis='x', labelsize=12)
ax_err.tick_params(axis='y', labelsize=12)
ax_err.legend(loc="best", ncol=2, fontsize=11)

# configure and save figure
fig_err.tight_layout()
fig_err_savename = "../images/err_vs_H_Tpeak.png"  # define savename with path
# fig_err.savefig(fig_err_savename, dpi=600)

## Plot incorrect fleet pairings vs time horizon for all climate objectives

This section performs plots the incorrect fleet pairings against the time horizon (Figure 1 in linked paper) for all climate objectives (peak and average temperatures). This section uses the data stored in `../data/MVMC/`. Uncomment the last line to save the figure to the `images` folder.

In [ ]:
metrics = ["F-RF", "F-AGWP", "F-AEGWP", "F-AGTP", "F-iAGTP & F-ATR", "F-GWP*", "F-EGWP*"]
markers = ["D", "o", "s", "h", "^", "*", "X"]
linestyles = ['-', '--', '--', '-.', ':', (0, (3, 1, 1, 1, 1, 1)), (0, (3, 1, 3, 1, 1, 1))]

# load data
H_lst = np.arange(5, 101, 5)
err_arr_maxT = np.load("../data/MVMC/tot_err_arr_maxT.npy")
err_arr_av20 = np.load("../data/MVMC/tot_err_arr_av20.npy")
err_arr_av50 = np.load("../data/MVMC/tot_err_arr_av50.npy")
err_arr_av100 = np.load("../data/MVMC/tot_err_arr_av100.npy")
err_arr_lst = [err_arr_maxT, err_arr_av20, err_arr_av50, err_arr_av100]

# initialise figure
fig = plt.figure(figsize=(9, 5))
gs = GridSpec(3, 3, figure=fig)
ax_mT = fig.add_subplot(gs[:, 0:2])
ax_20 = fig.add_subplot(gs[0, 2])
ax_50 = fig.add_subplot(gs[1, 2])
ax_100 = fig.add_subplot(gs[2, 2])

# plot
subplot_labels = ["a", "b", "c", "d"]
for j, ax in enumerate(fig.axes):
    for i in range(len(metrics)):
        ax.plot(H_lst, err_arr_lst[j][:, i], color=colors[i], linewidth=1.5, 
                label=metrics[i], linestyle=linestyles[i],
                marker=markers[i], markersize=5, markevery=(1,2))
    ax.set_xlim([-3, 103])
    ax.set_ylim([0, 8] if j == 0 else [-1, 8])

# adjust axes and labels
ax_mT.legend(loc="upper left", ncol=2, fontsize=11, handlelength=3)
ax_mT.set_ylabel("Total error frequency [%]", fontsize=12)
ax_mT.set_xlabel("Time horizon [year]", fontsize=12)
ax_20.set_xticklabels([])
ax_20.yaxis.tick_right()
ax_50.set_xticklabels([])
ax_50.set_ylabel("Total error frequency [%]", fontsize=12)
ax_50.yaxis.set_label_position("right")
ax_50.yaxis.tick_right()
ax_100.yaxis.tick_right()
ax_100.set_xlabel("Time horizon [year]", fontsize=12)

# add subplot labels
fig.text(0.61, 0.91, "a", fontsize=18)
fig.text(0.90, 0.91, "b", fontsize=18)
fig.text(0.90, 0.63, "c", fontsize=18)
fig.text(0.90, 0.35, "d", fontsize=18)

# configure and save plot
fig.tight_layout()
plt.subplots_adjust(wspace=0, hspace=0)
fig_savename = "../images/err_vs_H_full.png"  # define savename with path
# fig.savefig(fig_savename, dpi=600)